In [1]:
#import packages
import os 
from dotenv import load_dotenv
from alpha_client import get_stock_tools_openai
from agents import Agent, Runner , trace, Tool
from agents.mcp import MCPServerStdio
from datetime import datetime
from IPython.display import display, Markdown
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override= True)

True

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")

# free API key, no need to check for plan type
is_paid_polygon = False
is_realtime_polygon = False

print(is_paid_polygon)
print(is_realtime_polygon)


False
False


In [4]:
polygon_api_key = os.getenv("POLYGON_API_KEY")

# Always assume free plan
market_mcp = {"command": "uv","args": ["run", "market_server.py"]}

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

In [6]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

In [7]:
researcher_mcp_servers = [MCPServerStdio(params, cache_tools_list=True) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, cache_tools_list=True) for params in trader_mcp_server_params ]
mcp_servers = trader_mcp_servers + researcher_mcp_servers


### Now let's make a Researcher Agent to do market research

And turn it into a tool - 

In [8]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4o-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [9]:
#convert this agent to tool 
async def get_reseacher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
        tool_name = "Researcher",
        tool_description =  "This tool researches online for news and opportunities,\
            either based on specific requests to look into a certain stock,\
            or generally for notable financial news and opportunities.\
            Describe what kind of research you're looking for."
    )

In [19]:
#dry run of researcher tool 
research_question = "What is latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question)
display(Markdown(result.final_output))


Here are some of the latest news highlights regarding Amazon:

1. **New Partnership with The New York Times**: Amazon has formed a licensing agreement with The New York Times. This deal includes the use of the newspaper's editorial content on Amazon's platforms, which could enhance its content offerings.

2. **Shareholder Results Overview**: Recent reports indicate a continued drop in shareholder support regarding Amazon's environmental initiatives. This may affect how the company promotes its sustainability efforts moving forward.

3. **Unfounded Rumors**: There have been rumors circulating about Amazon potentially leaving the U.S. market. However, these claims have been fact-checked and deemed false by reputable sources.

4. **Stock Updates**: Amazon's stock continues to be closely monitored. Investors can find the latest quotes and financial information through platforms like Yahoo Finance and CNBC.

5. **Company Principles**: Amazon emphasizes its core principles of customer obsession, innovation, operational excellence, and long-term thinking in its communications and public relations.

For a deeper dive into these topics, you can visit [GeekWire](https://www.geekwire.com/amazon/) for more industry-specific news, or [Yahoo Finance](https://finance.yahoo.com/quote/AMZN/) for stock-related updates.

In [10]:
ellen_initial_strategy = " You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ellen").reset(ellen_initial_strategy)

display(Markdown(await read_accounts_resource("Ellen")))
display(Markdown(await read_strategy_resource("Ellen")))

{"name": "ellen", "balance": 10000.0, "strategy": " You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-06-12 09:08:18", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

 You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [11]:
agent_name = "Ellen"

#using MCP server to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f""" 
You are a trader that manages a portfolio of shares. Your name is {agent_name} and you account is under the name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, buy and sell shares.
Your investment strategy for you portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to preform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage you portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """ 
Use your tools to make decision about your portfolio.
Investigate the news and the market, make your decision, make the trades and respond with a summary of you actions.
"""

In [12]:
display(Markdown(instructions))

 
You are a trader that manages a portfolio of shares. Your name is Ellen and you account is under the name, Ellen.
You have access to tools that allow you to search the internet for company news, check stock prices, buy and sell shares.
Your investment strategy for you portfolio is:
 You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "ellen", "balance": 10000.0, "strategy": " You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-06-12 09:08:18", 10000.0], ["2025-06-12 09:08:47", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to preform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage you portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.


## finally run the trader 

In [13]:
for server in mcp_servers:
    await server.connect()

researcher_tools = await get_reseacher_tool(researcher_mcp_servers)
trader = Agent(
    name = agent_name,
    instructions=instructions,
    tools = [researcher_tools],
    mcp_servers= trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result= await Runner.run(trader, prompt)
display(Markdown(result.final_output))


### Summary of Actions

1. **Research Findings**:
   - The market sentiment is cautiously optimistic, with positive movements in tech stocks like Tesla and Intel, driven by good news and volatility.
   - Stocks are suitable for day trading due to their current performance and liquidity.

2. **Stock Prices**:
   - Tesla (TSLA): $326.43
   - Intel (INTC): $20.68

3. **Trades Executed**:
   - **Bought 20 shares of Tesla (TSLA)** at $327.08 each.
     - **Rationale**: Capitalizing on recent positive momentum and high trading volume suitable for quick gains.
   - **Bought 50 shares of Intel (INTC)** at $20.72 each.
     - **Rationale**: Targeting its strong performance and volatility for day trading.

4. **Portfolio Update**:
   - **Current Balance**: $2,422.27
   - **Holdings**:
     - Tesla (TSLA): 20 shares
     - Intel (INTC): 50 shares
   - **Total Portfolio Value**: $9,984.87
   - **Total Profit/Loss**: -$15.13 (value fluctuation)

### Next Steps
- Continue monitoring the market for further trading opportunities.
- Watch for any significant news or changes in stock performance, particularly in tech. Consider selling shares based on performance goals or new developments.

### Then go and look at the trace

http://platform.openai.com/traces

In [14]:
#look at the result for trading 
await read_accounts_resource(agent_name)

'{"name": "ellen", "balance": 2422.2748, "strategy": " You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"TSLA": 20, "INTC": 50}, "transactions": [{"symbol": "TSLA", "quantity": 20, "price": 327.08286, "timestamp": "2025-06-12 09:09:40", "rationale": "Taking advantage of recent positive momentum and high volatility for day trading."}, {"symbol": "INTC", "quantity": 50, "price": 20.72136, "timestamp": "2025-06-12 09:09:40", "rationale": "Intel\'s strong performance and volatility make it a suitable stock for quick trades."}], "portfolio_value_time_series": [["2025-06-12 09:08:18", 10000.0], ["2025-06-12 09:08:47", 10000.0], ["2025-06-12 09:09:40", 9986.9428], ["2025-06-12 09:09:40", 9984.874800000001], ["2025-06-12 09:10:55", 9984.874800000001]], "total_portfolio_value": 9984.874800000001, "total_profit_loss": -15.125199999998586}'

In [15]:
from traders import Trader

In [16]:
trader = Trader("Ellen")

In [17]:
result = await trader.run()

Error invoking MCP tool fetch: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.
Error invoking MCP tool fetch: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.
Error invoking MCP tool fetch: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.
Error invoking MCP tool fetch: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.
Error invoking MCP tool fetch: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.
Error invoking MCP tool fetch: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.
Error invoking MCP tool fetch: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.
Error invoking MCP tool fetch: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.
Error invoking MCP tool fetch: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.
Errored MCP tool result: meta=None content=[] 

[non-fatal] Tracing client error 400: {
  "error": {
    "message": "Invalid type for 'data[0].span_data.result': expected an array of strings, but got null instead.",
    "type": "invalid_request_error",
    "param": "data[0].span_data.result",
    "code": "invalid_type"
  }
}


In [18]:
await read_accounts_resource("Ellen")

'{"name": "ellen", "balance": 127.69370000000004, "strategy": " You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"TSLA": 20, "INTC": 30, "AAPL": 10, "NVDA": 5}, "transactions": [{"symbol": "TSLA", "quantity": 20, "price": 327.08286, "timestamp": "2025-06-12 09:09:40", "rationale": "Taking advantage of recent positive momentum and high volatility for day trading."}, {"symbol": "INTC", "quantity": 50, "price": 20.72136, "timestamp": "2025-06-12 09:09:40", "rationale": "Intel\'s strong performance and volatility make it a suitable stock for quick trades."}, {"symbol": "AAPL", "quantity": 10, "price": 199.17756, "timestamp": "2025-06-12 09:11:39", "rationale": "AAPL shows strong momentum and volatility for day trading."}, {"symbol": "INTC", "quantity": -20, "price": 20.63864, "timestamp": "2025-06-12 09:11:41", "rationale": "Selling partial holdings in INTC to free up cash for new opportunities."}, {"symbol": "NVDA", "quantity

In [19]:
display(Markdown(result))




<IPython.core.display.Markdown object>

How many tools did we use in total?

In [22]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("Ellen")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

TypeError: 'list' object is not callable